#### Setup inicial e leitura

In [32]:
import pandas as pd
import requests as req
from io import StringIO
from glob import glob as gb
import os

In [33]:
LOCAL_LOGS_FILEPATH = "/home/user/Faculdade/Sensores/logs"
MIN_LIMIT = -23000
MAX_LIMIT = 23000
os.mkdir("./temp")

FileExistsError: [Errno 17] Arquivo existe: './temp'

In [ ]:
def gravar_csv(df, filename):
    df.to_csv(f"./temp/{filename}.csv", sep="|")

In [ ]:
telemetry_files = gb(f"{LOCAL_LOGS_FILEPATH}/generation*")
parts = []

for f in telemetry_files:
    parts.append(pd.read_json(f))

telemetry_df = pd.concat(parts, ignore_index=True)


#### Filtragem e tratamentos

In [ ]:
tension = telemetry_df[
    (telemetry_df["valueType"] == "volts") &
    (telemetry_df["scenery"] == "NORMAL")
].reset_index(drop=True).copy()

tension["instant"] = pd.to_datetime(tension["instant"])

### API Pop. por zona

In [ ]:
API_URL = "https://pastebin.com/raw/Nz1aMUEM"
res = req.get(API_URL)

In [ ]:
pop_by_zone = pd.read_csv(StringIO(res.text))

In [ ]:
pop_by_zone["zone"] = pop_by_zone["zona"].str.upper()

In [ ]:
pop_by_zone = pop_by_zone.groupby("zone")["populacao"].sum()

### 1. Estouros por zona

In [ ]:
limits_by_zone = tension.groupby("zone").agg(
    BaixaTensao=("value", lambda x: x[x < MIN_LIMIT].count()),
    AltaTensao=("value", lambda x: x[x > MAX_LIMIT].count())
).sort_values(["AltaTensao", "BaixaTensao"], ascending=False)

limits_by_zone

,BaixaTensao,AltaTensao
zone,,
OESTE,2,1228
NORDESTE,1,1215
LESTE,3,1214
SUL,1,1205
CENTRO,2,1189
SUDESTE,1,1172
NORTE,0,1168


In [ ]:
limits_by_zone = limits_by_zone.merge(pop_by_zone, on="zone").rename(columns={"populacao": "PopulacaoTotal"}).sort_values("PopulacaoTotal", ascending=False)

In [ ]:
gravar_csv(limits_by_zone, "limits_by_zone")

#### 2. Estouros por horário

In [ ]:
def return_day_period(hour: int):
    period = None
    if 0 <= hour < 5:
        period = "Madrugada"
    elif 5 <= hour < 11:
        period = "Manhã"
    elif 11 <= hour < 14:
        period = "Meio dia"
    elif 14 <= hour < 16:
        period = "Início da tarde"
    elif 16 <= hour < 18:
        period = "Fim da tarde"
    else:
        period = "Noite"

    return period

In [ ]:
per_hour = tension[tension["value"] > MAX_LIMIT].copy()

per_hour["hour"] = per_hour["instant"].dt.strftime("%H").astype(int)
per_hour["day-period"] = per_hour["hour"].apply(return_day_period)

In [ ]:
per_hour = per_hour.groupby("day-period")["value"].count().reset_index().sort_values("value", ascending=False)

In [ ]:
per_hour

,day-period,value
3,Manhã,2074
5,Noite,2047
2,Madrugada,1698
4,Meio dia,1063
1,Início da tarde,826
0,Fim da tarde,683


In [ ]:
gravar_csv(per_hour, "per_hour")